In [43]:
import pandas as pd
import numpy as np
import calendar

In [44]:
def donnees_prevision():
    prevision= pd.read_excel("https://minio.lab.sspcloud.fr/nboukir/Donnees_AFP_prevision.xlsx")
    prevision=prevision[['Région','Optimiste','Laisser faire']].groupby('Région').mean() # On determine les températures moyennes par région, d'où le groupby.mean
    prevision= prevision.reset_index(level='Région')
    return prevision

In [45]:
def donnes_meteo_2020():
    meteo=pd.read_excel("https://minio.lab.sspcloud.fr/nboukir/meteo_2020.xlsx")
    def trans_jour(x):
        return x[8:10]
    
    def trans_mois(x):
        return x[5:7]

    def opti(reg) :
        return dict_prev_opti[reg]

    def lf (reg):
        return dict_prev_lf[reg]

    Dico_reg={'aquitaine':'Nouvelle-Aquitaine','bretagne':'Bretagne','alsace':'Grand Est','auvergne':'Auvergne Rohne Alpes',
                                     'franche-comte':'Bourgogne Franche Comté','ardeche':'Auvergne Rohne Alpes',
                                     'poitou-charentes':'Nouvelle-Aquitaine','rh-ne-alpes':'Auvergne Rohne Alpes',
                                     'provence-alpes-c-te-d-azur':'Provence-Alpes-Cote-d Azur','limousin': 'Nouvelle-Aquitaine',
                                     'lorraine':'Grand Est','picardie':'Hauts-de-France','champagne-ardenne':'Grand Est','nord-pas-de-calais':'Hauts-de-France', 
                                     'bourgogne':'Bourgogne Franche Comté','centre':'Centre-Val de Loire','ile-de-france':'Ile de France','languedoc-roussillon':'Occitanie','midi-pyrenees' :'Occitanie','normandie':'Normandie','pays-de-la-loire':'Pays de la Loire','corse':'to_remove','ile-de-re':'to_remove'}

    meteo=meteo.replace({'region': Dico_reg}) #On change le nom des régions pour que ce soit les mêmes entre tout les datasets.

    meteo=meteo.drop(meteo[(meteo['region']=='to_remove')].index,axis=0)  #On en profite pour retirer les régions non métropolitaines sur lesquels on a pas de données de consomation.

    meteo['TempMoy_Deg']=(meteo['TempMax_Deg']+meteo['TempMin_Deg'])/2 #On créer la colonne de température moyenne
    meteo=meteo[['region', 'day','TempMoy_Deg']] #On séléctionne seulement ces 3 colonnes in fine.
    meteo['day']=meteo['day'].astype(str)
    meteo['Jour']=meteo['day'].transform(trans_jour)
    meteo['Mois']=meteo['day'].transform(trans_mois) #On rejoute ces colonnes pour plus de lisibilité.

    #On intégre desormais les sénario optimiste et laisser faire dans la table meteo
    def donnees_prevision():
        prevision= pd.read_excel("https://minio.lab.sspcloud.fr/nboukir/Donnees_AFP_prevision.xlsx")
        prevision=prevision[['Région','Optimiste','Laisser faire']].groupby('Région').mean() # On determine les températures moyennes par région, d'où le groupby.mean
        prevision= prevision.reset_index(level='Région')
        return prevision

    prevision = donnees_prevision()

    dict_prev_opti=prevision[['Région','Optimiste']].to_dict('split')
    dict_prev_opti=dict(dict_prev_opti['data'])

    meteo['Temp_opti']=meteo['region'].transform(opti)+meteo['TempMoy_Deg']
    

    dict_prev_lf=prevision[['Région','Laisser faire']].to_dict('split')
    dict_prev_lf=dict(dict_prev_lf['data'])

    meteo['Temp_lf']=meteo['region'].transform(lf)+meteo['TempMoy_Deg']

    return meteo

In [46]:
def meteo_pic_conso():
    meteo_pic_conso=pd.read_excel("https://minio.lab.sspcloud.fr/nboukir/pic-journalier-consommation-brute.xlsx")
    meteo_pic_conso.columns = ['date','pic_journalier_consommation','temperature_moyenne', 'temperature_reference']
    meteo_pic_conso=meteo_pic_conso[['date','pic_journalier_consommation','temperature_moyenne']]
    return meteo_pic_conso

In [53]:
def conso_quotidienne():
    conso = pd.read_csv("https://odre.opendatasoft.com/explore/dataset/consommation-quotidienne-brute-regionale/download/?format=csv&disjunctive.code_insee_region=true&disjunctive.region=true&q=date_heure:%5B2019-12-31T23:00:00Z+TO+2020-12-31T22:59:59Z%5D&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separat" , sep = ';')

    def trans_jour(x):
        return x[8:10]
    
    def trans_mois(x):
        return x[5:7]

    def trans_annee(x):
        return x[0:4]

    def trans_timetofloat(x):
        return int(x[:2])+int(x[3:])/60.0

    def jour_semaine(x):
        int_mois=int(x[5:7])
        int_jour=int(x[8:10])
        return calendar.weekday(2020,int_mois,int_jour)
    
    conso.columns = ['date_heure' , 'date' , 'heure' , 'conde_insee' , 'region' , 'consommation_brute_gaz_GRTgaz' , 'status_GRTgaz' , 'consommation_brute_gaz_Terega' , 'status_Terega' , 'consommation_brute_gaz_totale' , 'consommation_brute_electricite_rte' , 'status_rte' , 'conso_brute_totale']
    conso['Année']=conso['date_heure'].apply(trans_annee)
    conso=conso[ (conso['Année']=='2020')] ##On selectionne l'année 2020
    conso=conso[['date','heure','region','consommation_brute_gaz_totale','consommation_brute_electricite_rte']] ## On séléctionne les colonnes qu'on veut garder


    conso.columns=['Date','Heure','Region','Consommation brute gaz (MW)','Consommation brute elecricite (MW)']

    conso['Consommation brute totale (MW)']=conso['Consommation brute gaz (MW)']+conso['Consommation brute elecricite (MW)']


    conso['Region']=conso['Region'].map({'Provence-Alpes-Côte d\'Azur':'Provence-Alpes-Cote-d Azur',
                             'Île-de-France':'Ile de France',
                             'Nouvelle-Aquitaine':'Nouvelle-Aquitaine',
                             'Occitanie':'Occitanie',
                             'Grand Est':'Grand Est',
                             'Pays de la Loire':'Pays de la Loire',
                             'Bretagne' : 'Bretagne',
                             'Hauts-de-France': 'Hauts-de-France',
                             'Centre-Val de Loire': 'Centre-Val de Loire',
                             'Normandie': 'Normandie',
                             'Bourgogne-Franche-Comté':'Bourgogne Franche Comté',
                             'Auvergne-Rhône-Alpes':'Auvergne Rohne Alpes'}) ## Les noms de régions ont une même appelation que les df precedent

## On donne des colonnes précise pour les jours et l'heure                      
    conso['Heure']=conso['Heure'].transform(trans_timetofloat)
    conso['Date']=conso['Date'].astype(str)
    conso['Jour']=conso['Date'].transform(trans_jour)
    conso['Mois']=conso['Date'].transform(trans_mois)
    conso['Jour semaine']=conso['Date'].transform(jour_semaine)
    conso['Mois']=conso['Mois'].astype(int)

## On implémente le dico des jours de la semaine pour plus de clareté

    conso['Jour semaine'] = conso['Jour semaine'].map({0:'Dimanche',1:'Lundi',2:'Mardi',3:'Mercredi',
                                                   4:'Jeudi',5:'Vendredi',6:'Samedi'})

    

## On écrit la liste région que l'on utilisera par la suite, notamment pour les graphiques

    Regions = ["Provence-Alpes-Cote-d Azur", 'Ile de France','Nouvelle-Aquitaine', 'Occitanie', 'Bourgogne Franche Comté','Grand Est', 'Pays de la Loire', 'Bretagne', 'Hauts-de-France','Centre-Val de Loire', 'Normandie', 'Auvergne Rohne Alpes']
    Pop_region = {'Provence-Alpes-Cote-d Azur':5055700,'Ile de France':12278200,'Nouvelle-Aquitaine':6000000,'Occitanie':5924000,'Grand Est':5511700,'Pays de la Loire':3801000,'Bretagne':3340400,'Hauts-de-France':5962700,'Centre-Val de Loire':2559100,'Normandie':3303500,'Bourgogne Franche Comté':2783000,'Auvergne Rohne Alpes':8032400}



## On pondére par le nombre d'habitant par régions pour pouvoir determiner des résultat à l'échelle de la France
    for reg in Regions :
        conso['Consogaz_ag (MW)']=conso['Consommation brute gaz (MW)']/Pop_region[reg]*67500000/12
        conso['Consoelec_ag (MW)']=conso['Consommation brute elecricite (MW)']/Pop_region[reg]*67500000/12
    

## On creer des colonnes de mesure d'energie en W pour plus de clareté pour la suite

    conso['Consommation brute totale (W)']=conso['Consommation brute totale (MW)']*1000000

    return conso

In [50]:
def conso_gaz():
    conso = conso_quotidienne()
    consogaz= conso[['Mois','Jour','Jour semaine','Heure','Region','Consommation brute gaz (MW)','Date','Consogaz_ag (MW)']]

    consogaz['Consommation brute gaz (MW)'].replace('', np.nan, inplace=True)

    consogaz['Consogaz_ag (MW)'].replace('', np.nan, inplace=True)

    consogaz.dropna(subset=['Consommation brute gaz (MW)','Consogaz_ag (MW)'], inplace=True)

    consogaz['Consommation brute gaz (W)']=consogaz['Consommation brute gaz (MW)']*1000000
    
    return consogaz


In [51]:
def conso_elec():
    conso = conso_quotidienne()
    consoelec=conso[['Mois','Jour','Jour semaine','Heure','Region','Consommation brute elecricite (MW)','Date','Consoelec_ag (MW)']]

    consoelec['Consommation brute elecricite (MW)'].replace('', np.nan, inplace=True)
    consoelec['Consoelec_ag (MW)'].replace('', np.nan, inplace=True)

    consoelec.dropna(subset=['Consommation brute elecricite (MW)','Consoelec_ag (MW)'], inplace=True)

    consoelec['Consommation brute elec (W)']=consoelec['Consommation brute elecricite (MW)']*1000000

    return consoelec